In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Importing libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import seaborn as sns
import scipy.stats as stats
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, f1_score, recall_score
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Reading Dataset

In [ ]:
data = pd.read_parquet('/content/drive/MyDrive/IE/Capstone/transactions_dataset.parquet')

In [ ]:
data.head()

# Churn Exploration

In this section, we explore the definition of churn.

We came up with 5 approaches:
* Confidence interval
* K-means
* Confidence interval + K-means
* K-means + purchase interval
* Windows (applied solution)

## Confidence interval calculation

The primary concept behind using a confidence interval is to determine where the recency of each client's purchase falls on a normal distribution curve. If the purchase intervals of clients follow a normal distribution, we can calculate the position of the recency within this curve. When the recency lies beyond the right tail of the 95% confidence interval, it indicates that the likelihood of such a recency occurring is less than 2.5%. This suggests that the customer has most likely churned.

However, after conducting a normality test on the purchase behaviour of our customers, we found that it does not follow a normal distribution. As a result, we cannot apply this approach to assess customer churn accurately.

In [ ]:
df = pd.DataFrame(data)
df['date_order'] = pd.to_datetime(df['date_order'])

# keep 1 each day
df = df.drop_duplicates(subset=['client_id', 'date_order'])

# order by client_id and date_order
df = df.sort_values(by=['client_id', 'date_order'])

# calculate interval
df['purchase_interval'] = df.groupby('client_id')['date_order'].diff().dt.days

# delete null
purchase_intervals = df.dropna(subset=['purchase_interval'])

In [ ]:
purchase_intervals.head()

In [ ]:
# Function to check normality for individual customers
def check_customer_normality(df, client_id):
    customer_data = df[df['client_id'] == client_id]['purchase_interval']
    if len(customer_data) < 3:  # Shapiro-Wilk test requires at least 3 data points
        return np.nan, np.nan
    shapiro_test = stats.shapiro(customer_data)
    return shapiro_test.statistic, shapiro_test.pvalue

# Apply this function to each customer
results = []
for client in purchase_intervals['client_id'].unique():
    statistic, pvalue = check_customer_normality(purchase_intervals, client)
    results.append({'client_id': client, 'statistic': statistic, 'pvalue': pvalue})

results_df = pd.DataFrame(results)

# proportion
alpha = 0.05
results_df['is_normal'] = results_df['pvalue'].apply(lambda p: p > alpha)
normal_customers = results_df['is_normal'].sum()
total_customers = results_df['client_id'].nunique()
normal_ratio = normal_customers / total_customers

print(f"Number of customers following normal distribution: {normal_customers}")
print(f"Total number of customers: {total_customers}")
print(f"Proportion of customers following normal distribution: {normal_ratio:.2f}")

# result
print(results_df)


## K-means

K-means is able to cluster the customers according to their puchase behaviour, so by using k-means, we might be able to find a cluster that implies high probability of churn.

Though we did find a cluster with extremely high recency, the cluster also shows characteristics of low clv and low frequency, which means that those customers in the cluster are the ones that have purchased only once or twice, so we were not able to detect the churn of high value customers.

### Finding the best cluster number

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/IE/Capstone/client.csv')

In [ ]:
#reseting indexes
data=data.set_index('Unnamed: 0',drop=True)
data=data.reset_index(drop=True)

In [ ]:
data.info()

In [ ]:
df = pd.DataFrame(data)

# Select relevant columns
features = ['recency', 'frequency', 'monetary', 'avg_days_between_purchases', 'returns',
            'unique_products', 'total_quantity', 'purchase_interval_variance', 'lifespan']

# Handle missing values
df = df.dropna(subset=features)

# Standardize the data
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df[features])

### Step 3: K-means Clustering

# Determine the optimal number of clusters using the elbow method
inertia = []
range_n_clusters = range(1, 13)

for n_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(scaled_features)
    inertia.append(kmeans.inertia_)

# Plot the elbow curve
plt.figure(figsize=(8, 4))
plt.plot(range_n_clusters, inertia, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal Number of Clusters')
plt.grid(True)
plt.show()

### Cluster Analysis

The clustering analysis is done in `Dataiku`, the following is just an example showing how the clusters might look like.

In [ ]:
# Fit K-means with the optimal number of clusters
optimal_n_clusters = 5
kmeans = KMeans(n_clusters=optimal_n_clusters, random_state=0)
df['cluster'] = kmeans.fit_predict(scaled_features)

### Step 4: Cluster Analysis

# Add cluster labels to the original DataFrame
df['cluster'] = kmeans.labels_

# Analyze clusters
cluster_summary = df.groupby('cluster')[features].mean()

# Check the number of clients in each cluster
cluster_counts = df['cluster'].value_counts()

# Plot cluster distribution
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='cluster')
plt.xlabel('Cluster')
plt.ylabel('Number of Clients')
plt.title('Number of Clients in Each Cluster')
plt.show()

## K-means + Confidence Interval

In order to detect the churn status of the customers with high value, we tried to **conduct the confidence interval within the high value clusters**. However, those clusters did not pass through the normality test and the approach was abandoned.

In [ ]:
clustered = pd.read_csv('/content/drive/MyDrive/IE/Capstone/client_correct_clustered.csv')

In [ ]:
clustered.head()

In [ ]:
# cluster_0: tier_3; cluster_1: churned; cluster_2: tier_1; cluster_3: at_risk; cluster_4: tier_2
clustered['cluster_labels'].value_counts()

#### Normality test

In [ ]:
data = pd.read_parquet('/content/drive/MyDrive/IE/Capstone/transactions_dataset.parquet')

In [ ]:
# Initialize a DataFrame with your data
df = data
df['date_order'] = pd.to_datetime(df['date_order'])

# Keep only one transaction per client per day
df = df.drop_duplicates(subset=['client_id', 'date_order'])

# Order by client_id and date_order
df = df.sort_values(by=['client_id', 'date_order'])

# Calculate purchase intervals
df['purchase_interval'] = df.groupby('client_id')['date_order'].diff().dt.days

# Delete rows with null purchase_interval
purchase_intervals = df.dropna(subset=['purchase_interval'])

# Merge with clustered DataFrame
clustered = pd.merge(purchase_intervals, clustered[['client_id', 'cluster_labels']], on='client_id')

# Ensure 'purchase_interval' column has no NaN values after merging
clustered = clustered.dropna(subset=['purchase_interval'])

# Function to check normality for individual customers
def check_customer_normality(df, client_id):
    customer_data = df[df['client_id'] == client_id]['purchase_interval'].dropna()
    if len(customer_data) < 3:  # Shapiro-Wilk test requires at least 3 data points
        return np.nan, np.nan
    shapiro_test = stats.shapiro(customer_data)
    return shapiro_test.statistic, shapiro_test.pvalue

# Initialize a list to hold results for all specified clusters
results = []

# List of clusters to analyze
clusters_to_analyze = ['cluster_1', 'cluster_3', 'cluster_4']

for cluster in clusters_to_analyze:
    # Filter the dataframe for the current cluster
    cluster_df = clustered[clustered['cluster_labels'] == cluster]

    # Apply normality test to each customer in the current cluster
    for client in cluster_df['client_id'].unique():
        statistic, pvalue = check_customer_normality(cluster_df, client)
        results.append({'client_id': client, 'cluster_label': cluster, 'statistic': statistic, 'pvalue': pvalue})

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Calculate the proportion of customers following a normal distribution for each cluster
alpha = 0.05
results_df['is_normal'] = results_df['pvalue'].apply(lambda p: p > alpha)

# Proportion of normal customers per cluster
for cluster in clusters_to_analyze:
    cluster_results = results_df[results_df['cluster_label'] == cluster]
    normal_customers = cluster_results['is_normal'].sum()
    total_customers = cluster_results['client_id'].nunique()
    normal_ratio = normal_customers / total_customers if total_customers > 0 else 0
    print(f"Cluster {cluster} - Number of customers following normal distribution: {normal_customers}")
    print(f"Cluster {cluster} - Total number of customers: {total_customers}")
    print(f"Cluster {cluster} - Proportion of customers following normal distribution: {normal_ratio:.2f}")

# Print overall result
print(results_df)


## K-means + Average purchasing days

Knowing that confidence interval is not available in this case, we tried defining churn with K-means and average purchasing days.

The approach is as follows:
- apply churned for customers belonging to the cluster with extremely high recency and CLV.
- apply churned for customers that recency is 3 times larger than their average purchasing day.

This approach is able to customise the definition of churn according to different customers and also consider the customer value.

In [ ]:
clustered = pd.read_csv('/content/drive/MyDrive/IE/Capstone/client_correct_clustered.csv')

In [ ]:
clustered.info()

In [ ]:
# Set a threshold multiplier. If recency exceeds 3 times the average purchase interval, the customer is at risk of churn
threshold_multiplier = 3

# Define churn risk
clustered['churn_risk'] = clustered['recency'] > (threshold_multiplier * clustered['avg_days_between_purchases'])

# Print results
print(clustered[['client_id', 'recency', 'avg_days_between_purchases', 'churn_risk']])

# Count the number of customers at risk of churn
churn_customers = clustered['churn_risk'].sum()
total_customers = clustered['client_id'].nunique()
churn_ratio = churn_customers / total_customers

print(f"Number of customers at risk of churn: {churn_customers}")
print(f"Total number of customers: {total_customers}")
print(f"Proportion of customers at risk of churn: {churn_ratio:.2f}")


In [ ]:
# Create the 'churn' column based on the condition
clustered.loc[clustered['cluster_labels']=='cluster_0','churn_risk'] = True

clustered.reset_index(drop=True,inplace=True)
clustered.head()

# Count the number of customers at risk of churn
churn_customers = clustered['churn_risk'].sum()
total_customers = clustered['client_id'].nunique()
churn_ratio = churn_customers / total_customers

print(f"Number of customers at risk of churn: {churn_customers}")
print(f"Total number of customers: {total_customers}")
print(f"Proportion of customers at risk of churn: {churn_ratio:.2f}")

In [ ]:
clustered[clustered['cluster_labels']=='cluster_1']

In [ ]:
churn_customers = clustered.loc[clustered['cluster_labels']=='cluster_4','churn_risk'].sum()
total_customers = clustered[clustered['cluster_labels']=='cluster_4']['client_id'].count()
churn_ratio = churn_customers / total_customers
print(f"Number of customers at risk of churn: {churn_customers}")
print(f"Total number of customers: {total_customers}")
print(f"Proportion of customers at risk of churn: {churn_ratio:.2f}")


## Windows approach

Though the K-means + average purchasing days approach can define churn differently according to the customer purchase behaviour, it is possible to cause data leakage when training a model. Therefore, we decided to use the window approach for the model.

The idea is seperating the dataset into different windows, we used 2 months as the time period in this case. Then we assign the value of 1, if the customer has purchased during the 2 months, and 0, if the has not purchased. We obtained a new dataframe containing the customer behaviour and we're going to use it for prediction.


In [ ]:
# reading the dataset
data = pd.read_parquet('/content/drive/MyDrive/IE/Capstone/transactions_dataset.parquet')

# datetime
data['date_order'] = pd.to_datetime(data['date_order'])

# dataset for frequency calculation
data_unique = data.drop_duplicates(subset=['client_id', 'date_order'])

# generate 2-month window, starting from 0
min_date = data_unique['date_order'].min()
data_unique['window'] = ((data_unique['date_order'].dt.year - min_date.year) * 6 +
                         (data_unique['date_order'].dt.month - min_date.month) // 2)

# generate client_behaviour
client_behaviour = data_unique.groupby(['client_id', 'window']).size().unstack(fill_value=0)

In [ ]:
client_behaviour.columns = [0,1,2,3,4,5,6,7,8,9,10,11,12]
# define purchase behaviour
def process_window_12(value):
    return 0 if value == 0 else 1

# assign values to each of the columns
for column in client_behaviour.columns:
    client_behaviour[column] = client_behaviour[column].apply(process_window_12)
client_behaviour.head()

In [ ]:
client_behaviour[12].value_counts()

In [ ]:
client_behaviour = pd.read_csv( '/content/drive/MyDrive/IE/Capstone/client_behaviour.csv', index_col=0)

# Churn Prediction Model

To decrease the churn rate of the customers and increase customer lifetime value, we developed a churn model based on the window dataset that we generated for customer behaviour analysis.

We tried 2 approaches:
- We created a dataset without the transactions happening in the lastest 30 days, and generated a client dataset based on it. The churn status is calculated in both datasets (with and without the latest 30 days transactions), and we tried to predict the changes of churn status.
- We used the purchased-or-not window dataset to predict whether a customer is going to purchase or not in the next two months.

## First Approach - based on client dataset

As mentioned,in the first approach, we tried using a client dataset without the transactions happening in the latest 30 days. Though without the 30 days of transactions, the RFM, CLV and other values will change accordingly and prevent data leakage, the definition of churn here is rather robust and hard to change over time, which results in the merely 18% of the changes of status during the one month period.

Additionally, using a summary of the whole history for prediction is not actually tracking customer behaviour evolution, for which we developed the second approach.

### Dataset without latest 30 days' transactions

In [ ]:
# Converting datetime format
data = pd.read_parquet('/content/drive/MyDrive/IE/Capstone/transactions_dataset.parquet')
data['date_order'] = pd.to_datetime(data['date_order'])
data['date_invoice'] = pd.to_datetime(data['date_invoice'])

# Determine the latest date in the data
latest_date_order = data['date_order'].max()

# Calculate the cutoff date
cutoff_date = latest_date_order - pd.Timedelta(days=30)

# Filter the data to exclude transactions from the last 30 days
data = data[data['date_order'] < cutoff_date]

# Display the filtered data
print(data.head())

# Save the filtered dataset if needed
# filtered_data.to_parquet('/path/to/save/filtered_data.parquet')


### Client profile for prediction

In [ ]:
# Latest date in the dataset
latest_date = data['date_order'].max()

# Drop duplicate transactions per customer per day
unique_transactions = data.drop_duplicates(subset=['client_id', 'date_order'])

# Aggregate the data to calculate RFM metrics
rfm = data.groupby('client_id').agg({
    'date_order': lambda x: (latest_date - x.max()).days,
    'sales_net': 'sum'
}).reset_index()

# Calculate frequency from unique_transactions
frequency = unique_transactions.groupby('client_id')['date_order'].count()

# Merge frequency into the RFM dataframe
rfm = rfm.merge(frequency, on='client_id')

# Rename columns for clarity
rfm = rfm.rename(columns={
    'date_order': 'recency',
    'sales_net': 'monetary'
})
df = data.drop_duplicates(subset=['client_id', 'date_order'])
df = df.sort_values(by=['client_id', 'date_order'])

# calculate interval
df['purchase_interval'] = df.groupby('client_id')['date_order'].diff().dt.days
avg_days_between_purchases = df.groupby('client_id')['purchase_interval'].mean().reset_index(name='avg_days_between_purchases')
variance_df = df.groupby('client_id')['purchase_interval'].var().reset_index(name='purchase_interval_variance')
variance_df['purchase_interval_variance'] = variance_df['purchase_interval_variance'].fillna(0)

# Replace NaN with 0 for customers with only one purchase
most_common_channel = data.groupby('client_id')['order_channel'].agg(lambda x: x.mode()[0]).reset_index(name='most_common_order_channel')

# Most Common Order Channel
most_common_channel = data.groupby('client_id')['order_channel'].agg(lambda x: x.mode()[0]).reset_index(name='most_common_order_channel')

# Number of returns
returns = data[data['sales_net'] < 0].groupby('client_id').size().reset_index(name='returns')
returns['returns'] = returns['returns'].fillna(0)

# Most Purchased Product
most_purchased_product = data.groupby(['client_id', 'product_id']).size().reset_index(name='count')
most_purchased_product = most_purchased_product.loc[most_purchased_product.groupby('client_id')['count'].idxmax()].drop(columns='count')

# Number of Unique Products Purchased
unique_products = data.groupby('client_id')['product_id'].nunique().reset_index(name='unique_products')

# Brach
most_common_branch = data.groupby('client_id')['branch_id'].agg(lambda x: x.mode()[0]).reset_index(name='most_common_branch')

# total quantity
total_quantity = data.groupby('client_id')['quantity'].sum().reset_index(name='total_quantity')

# First Purchase date
first_purchase_date = data.groupby('client_id')['date_order'].min().reset_index(name='first_purchase_date')

last_purchase_date = data.groupby('client_id')['date_order'].max().reset_index(name='last_purchase_date')
last_invoice_date = data.groupby('client_id')['date_invoice'].max().reset_index(name='last_invoice_date')
#CLV
# value
avg_purchase_value = data.groupby('client_id')['sales_net'].mean()
# lifespan
customer_lifespan = (data.groupby('client_id')['date_order'].max() - data.groupby('client_id')['date_order'].min()).dt.days / 365
# calculation
clv = avg_purchase_value * frequency * customer_lifespan

clv_df = pd.DataFrame({'client_id': clv.index, 'CLV': clv.values})
print(clv_df.head())
final_df = first_purchase_date \
    .merge(last_purchase_date, on='client_id') \
    .merge(last_invoice_date, on='client_id')\
    .merge(rfm, on='client_id') \
    .merge(avg_days_between_purchases, on='client_id') \
    .merge(most_common_channel, on='client_id') \
    .merge(returns, on='client_id', how='left') \
    .merge(most_purchased_product, on='client_id') \
    .merge(unique_products, on='client_id') \
    .merge(most_common_branch, on='client_id') \
    .merge(total_quantity, on='client_id') \
    .merge(variance_df, on='client_id')\
    .merge(customer_lifespan, on='client_id')\
    .merge(clv_df, on='client_id')

final_df=final_df.rename(columns={'date_order':'lifespan','date_order_x':'recency','date_order_y':'frequency'})
# Fill NA values in returns with 0
final_df['returns'] = final_df['returns'].fillna(0)
final_df['avg_days_between_purchases'] = final_df['avg_days_between_purchases'].fillna(0)

# Display the final DataFrame
final_df.head()

final_df.to_csv('/content/drive/MyDrive/IE/Capstone/client_prediction.csv')

### Feature engineering

In [ ]:
final_df = pd.read_csv('/content/drive/MyDrive/IE/Capstone/client_prediction.csv')
clustered = pd.read_csv('/content/drive/MyDrive/IE/Capstone/client_correct_clustered.csv')

In [ ]:
clustered.head()

In [ ]:
final_df = final_df.merge(clustered[['client_id','cluster_labels']],on='client_id')

In [ ]:
# Set a threshold multiplier. If recency exceeds 3 times the average purchase interval, the customer is at risk of churn
threshold_multiplier = 3

# Define churn risk
final_df['churn_risk'] = final_df['recency'] > (threshold_multiplier * final_df['avg_days_between_purchases'])

# Print results
print(final_df[['client_id', 'recency', 'avg_days_between_purchases', 'churn_risk']])

# Count the number of customers at risk of churn
churn_customers = final_df['churn_risk'].sum()
total_customers = final_df['client_id'].nunique()
churn_ratio = churn_customers / total_customers

print(f"Number of customers at risk of churn: {churn_customers}")
print(f"Total number of customers: {total_customers}")
print(f"Proportion of customers at risk of churn: {churn_ratio:.2f}")


In [ ]:
# Set a threshold multiplier. If recency exceeds 3 times the average purchase interval, the customer is at risk of churn
threshold_multiplier = 3

# Define churn risk
clustered['churn_risk'] = clustered['recency'] > (threshold_multiplier * clustered['avg_days_between_purchases'])

# Print results
print(clustered[['client_id', 'recency', 'avg_days_between_purchases', 'churn_risk']])

# Count the number of customers at risk of churn
churn_customers = clustered['churn_risk'].sum()
total_customers = clustered['client_id'].nunique()
churn_ratio = churn_customers / total_customers

print(f"Number of customers at risk of churn: {churn_customers}")
print(f"Total number of customers: {total_customers}")
print(f"Proportion of customers at risk of churn: {churn_ratio:.2f}")

In [ ]:
final_df.head()

In [ ]:
# Adding the result after 30 days
final_df = final_df.merge(clustered[['client_id','churn_risk']],on='client_id')
final_df.rename(columns={'churn_risk_y':'churn_result','churn_risk_x':'churn_status'},inplace=True)

In [ ]:
final_df.head()

### Model Training and Evaluation

In [ ]:
pip install xgboost


In [ ]:
clustered['last_purchase_date'].max()

In [ ]:
final_df['last_purchase_date'].max()

In [ ]:
# Exclude 'cluster_labels' and other non-predictive columns
X = final_df.drop(columns=['client_id', 'cluster_labels', 'churn_result', 'first_purchase_date', 'last_purchase_date', 'last_invoice_date','product_id','most_common_branch'])
y = final_df['churn_result']

# Encode categorical variables
for column in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Print shapes for verification
print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")


In [ ]:
# Encode categorical variables
for column in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column])


In [ ]:
X.columns

In [ ]:
y.value_counts()

In [ ]:
# Calculate class weights to handle class imbalance
neg, pos = np.bincount(y_train)
scale_pos_weight = neg / pos

# Initialize the XGBoost classifier
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', scale_pos_weight=scale_pos_weight)

# Define the parameter grid for random search
param_dist = {
    'n_estimators': np.arange(50, 300, 50),
    'max_depth': np.arange(3, 10, 1),
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'min_child_weight': [1, 2, 3, 4, 5]
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=10,  # Number of parameter settings sampled
    scoring='roc_auc',  # Use recall as the scoring metric
    cv=4,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit RandomizedSearchCV
random_search.fit(X_train, y_train)

# Get the best parameters
best_params = random_search.best_params_
print("Best Parameters:", best_params)

# Train the final model with the best parameters
best_model = XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss', scale_pos_weight=scale_pos_weight)
best_model.fit(X_train, y_train)


In [ ]:
# Make predictions
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

# Adjust threshold to favor recall
threshold = 0.3  # Lower the threshold to favor recall
y_pred = (y_pred_proba >= threshold).astype(int)

# Evaluate the XGBoost model
print("XGBoost Model Classification Report:")
print(classification_report(y_test, y_pred))
print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_proba):.2f}")

# Calculate accuracy for the XGBoost model
xgboost_accuracy = accuracy_score(y_test, y_pred)
print(f"XGBoost Model Accuracy: {xgboost_accuracy:.2f}")

# Calculate AUC-ROC for the XGBoost model
xgboost_auc_roc = roc_auc_score(y_test, y_pred_proba)
print(f"XGBoost Model AUC-ROC: {xgboost_auc_roc:.2f}")

# Calculate Recall for the XGBoost model
xgboost_recall = recall_score(y_test, y_pred)
print(f"XGBoost Model Recall: {xgboost_recall:.2f}")

In [ ]:
# Baseline predictions: predict no change in churn status
y_baseline_pred = X_test['churn_status'].copy()

# Calculate accuracy for baseline model
baseline_accuracy = accuracy_score(y_test, y_baseline_pred)

# Calculate the number of correct predictions in the baseline model
same = (y_test == y_baseline_pred).sum()
total = len(y_test)
baseline_accuracy_manual = same / total

print(f"Baseline Accuracy (using accuracy_score): {baseline_accuracy:.2f}")
print(f"Baseline Accuracy (manual calculation): {baseline_accuracy_manual:.2f}")

# Calculate other metrics for the baseline model
print("Baseline Model Classification Report:")
print(classification_report(y_test, y_baseline_pred))


In [ ]:
comparison = {
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC'],
    'Baseline Model': [
        baseline_accuracy,
        classification_report(y_test, y_baseline_pred, output_dict=True)['weighted avg']['precision'],
        classification_report(y_test, y_baseline_pred, output_dict=True)['weighted avg']['recall'],
        classification_report(y_test, y_baseline_pred, output_dict=True)['weighted avg']['f1-score'],
        'N/A'  # AUC-ROC is not applicable for the baseline model since probabilities are not used
    ],
    'XGBoost Model': [
        xgboost_accuracy,
        classification_report(y_test, y_pred, output_dict=True)['weighted avg']['precision'],
        classification_report(y_test, y_pred, output_dict=True)['weighted avg']['recall'],
        classification_report(y_test, y_pred, output_dict=True)['weighted avg']['f1-score'],
        xgboost_auc_roc
    ]
}

comparison_df = pd.DataFrame(comparison)
print(comparison_df)


The following part is model evaluation based on the important clusters.

In [ ]:
important_customers = final_df[final_df['cluster_labels'].isin(['cluster_1','cluster_4','cluster_3'])]
X_imp = important_customers.drop(columns=['client_id', 'cluster_labels', 'churn_result', 'first_purchase_date', 'last_purchase_date', 'last_invoice_date','product_id','most_common_branch'])
y_imp = important_customers['churn_result']

In [ ]:
# Encode categorical variables
for column in X_imp.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X_imp[column] = le.fit_transform(X_imp[column])

In [ ]:
# Make predictions
y_pred = best_model.predict(X_imp)
y_pred_proba = best_model.predict_proba(X_imp)[:, 1]

# Evaluate the XGBoost model
print("XGBoost Model Classification Report:")
print(classification_report(y_imp, y_pred))
print(f"AUC-ROC: {roc_auc_score(y_imp, y_pred_proba):.2f}")

# Calculate accuracy for the XGBoost model
xgboost_accuracy = accuracy_score(y_imp, y_pred)
print(f"XGBoost Model Accuracy: {xgboost_accuracy:.2f}")

# Calculate AUC-ROC for the XGBoost model
xgboost_auc_roc = roc_auc_score(y_imp, y_pred_proba)
print(f"XGBoost Model AUC-ROC: {xgboost_auc_roc:.2f}")

In [ ]:
# Baseline predictions: predict no change in churn status
y_baseline_pred = X_imp['churn_status'].copy()

# Calculate accuracy for baseline model
baseline_accuracy = accuracy_score(y_imp, y_baseline_pred)

# Calculate the number of correct predictions in the baseline model
same = (y_imp == y_baseline_pred).sum()
total = len(y_test)
baseline_accuracy_manual = same / total

print(f"Baseline Accuracy (using accuracy_score): {baseline_accuracy:.2f}")
print(f"Baseline Accuracy (manual calculation): {baseline_accuracy_manual:.2f}")

# Calculate other metrics for the baseline model
print("Baseline Model Classification Report:")
print(classification_report(y_imp, y_baseline_pred))


In [ ]:
comparison = {
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC'],
    'Baseline Model': [
        baseline_accuracy,
        classification_report(y_imp, y_baseline_pred, output_dict=True)['weighted avg']['precision'],
        classification_report(y_imp, y_baseline_pred, output_dict=True)['weighted avg']['recall'],
        classification_report(y_imp, y_baseline_pred, output_dict=True)['weighted avg']['f1-score'],
        'N/A'  # AUC-ROC is not applicable for the baseline model since probabilities are not used
    ],
    'XGBoost Model': [
        xgboost_accuracy,
        classification_report(y_imp, y_pred, output_dict=True)['weighted avg']['precision'],
        classification_report(y_imp, y_pred, output_dict=True)['weighted avg']['recall'],
        classification_report(y_imp, y_pred, output_dict=True)['weighted avg']['f1-score'],
        xgboost_auc_roc
    ]
}

comparison_df = pd.DataFrame(comparison)
print(comparison_df)


In [ ]:
# Assume y_true and y_pred_proba are given
y_true = y_imp
y_pred_proba = best_model.predict_proba(X_imp)[:, 1]

# Compute deciles
deciles = pd.qcut(y_pred_proba, 10, labels=False)
df = pd.DataFrame({'decile': deciles, 'true_label': y_true, 'pred_proba': y_pred_proba})

# Calculate lift for each decile
lift_table = df.groupby('decile').apply(
    lambda x: pd.Series({
        'num_customers': x.shape[0],
        'num_positives': x['true_label'].sum(),
        'positive_rate': x['true_label'].mean(),
        'lift': x['true_label'].mean() / y_true.mean()
    })
).reset_index()

# Calculate top decile lift
top_decile_lift = lift_table.loc[lift_table['decile'] == 9, 'lift'].values[0]

print(lift_table)
print(f"Top Decile Lift: {top_decile_lift:.2f}")

# Detailed lift table interpretation
for index, row in lift_table.iterrows():
    print(f"Decile {int(row['decile'])}:")
    print(f"  Number of customers: {row['num_customers']}")
    print(f"  Number of positives: {row['num_positives']}")
    print(f"  Positive rate: {row['positive_rate']:.4f}")
    print(f"  Lift: {row['lift']:.4f}")

## Second Approach - Purchase Windows

In order to consider the customer behaviours evolution, we used the window approach to predict whether a customer will churn in the next 2 months.

### Reading the dataset

In [ ]:
client_behaviour = pd.read_csv('/content/drive/MyDrive/IE/Capstone/client_behaviour.csv',index_col='client_id')

In [ ]:
client_behaviour.head()

### Feature Engineering

Instead of recording the number of purchases made during the windows, we decided to simplify it to bought(1) or not(0) which facilitates our training using rolling windows.

In [ ]:
client_behaviour.columns = [0,1,2,3,4,5,6,7,8,9,10,11,12]
def process_window_12(value):
    return 0 if value == 0 else 1
for column in client_behaviour.columns:
    client_behaviour[column] = client_behaviour[column].apply(process_window_12)
client_behaviour.head()

In [ ]:
client_behaviour[12].value_counts()

### Model Training and Evaluation

In [ ]:
# Initialize result storage
results = {
    'window': [],
    'f1_score': [],
    'roc_auc': [],
    'accuracy': [],
}
# Set initial window size and maximum window size
initial_window_size = 3
max_window_size = client_behaviour.shape[1]

# Define parameter distribution for random search
param_dist = {
    'n_estimators': np.arange(50, 300, 50),
    'max_depth': np.arange(3, 10, 1),
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'min_child_weight': [1, 2, 3, 4, 5]
}

feature_importance_df = pd.DataFrame()
best_params = None

for start_window in range(0, max_window_size - initial_window_size):
    train_windows = list(range(0, start_window + initial_window_size))
    test_window = start_window + initial_window_size

    # Training and testing sets
    X_train = client_behaviour[train_windows]
    y_train = client_behaviour[test_window]
    X_test = client_behaviour[train_windows]
    y_test = client_behaviour[test_window]

    # Feature engineering
    feature_names = [str(i) for i in range(X_train.shape[1])]
    X_train.columns = feature_names
    X_test.columns = feature_names

    # Initialize XGBoost classifier
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

    # Initialize random search
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_dist,
        n_iter=50,  # Number of parameter combinations, adjust as needed
        scoring='f1',
        cv=4,
        verbose=2,
        random_state=327,
        n_jobs=-1
    )

    # Perform random search
    random_search.fit(X_train, y_train)

    # Get best parameters
    best_params = random_search.best_params_
    print(f"Best Parameters for window {test_window}: {best_params}")

    # Train final model with best parameters
    best_model = XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss')
    best_model.fit(X_train, y_train)

    # Make predictions
    y_pred = best_model.predict(X_test)
    y_pred_proba = best_model.predict_proba(X_test)[:, 1]

    # Evaluate model
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    accuracy = accuracy_score(y_test, y_pred)

    # Store results
    results['window'].append(test_window)
    results['f1_score'].append(f1)
    results['roc_auc'].append(roc_auc)
    results['accuracy'].append(accuracy)

    # Store feature importance
    feature_importances = best_model.feature_importances_
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': feature_importances,
        'window': test_window
    })
    feature_importance_df = pd.concat([feature_importance_df, importance_df], axis=0)

    # Output results for each iteration
    print(f"Window: {test_window}")
    print(f"F1 Score: {f1:.2f}")
    print(f"AUC-ROC: {roc_auc:.2f}")
    print(f"Accuracy: {accuracy:.2f}")
    # Convert results to DataFrame
results_df = pd.DataFrame(results)
print(results_df)


In [ ]:
results_df

As shown, the model suffers a degradation in window 12, we assumed that there should be a large sample difference and the following part justified our assumption. The positive ratio is much lower in the last 2 months and it affected the model performance.

In [ ]:
# checking the distribution of each window
for start_window in range(0, max_window_size - initial_window_size):
    test_window = start_window + initial_window_size
    y_test = client_behaviour[test_window]
    positive_ratio = y_test.sum() / len(y_test)
    print(f"Window {test_window}: Positive ratio = {positive_ratio:.2f}")

Here we're predicting whether the customer will pay or not next 2 months.

In [ ]:
# Prepare input data for window 13 (assuming features from windows 1 to 12)
X_window_13 = client_behaviour[list(range(1, 13))]  # Features from windows 1 to 12
feature_names_window_13 = [str(i) for i in range(X_window_13.shape[1])]
X_window_13.columns = feature_names_window_13

# Make predictions for window 13 using the extended model
predictions_window_13 = best_model.predict(X_window_13)
predictions_proba_window_13 = best_model.predict_proba(X_window_13)[:, 1]

# Optionally, you can store predictions_window_13 and predictions_proba_window_13 for further analysis or output.

print(f"Predictions for window 13:")
print(predictions_window_13)
print(predictions_proba_window_13)


### Merging the result of predictions with client dataset

In this part, we merge the two datasets to prepare it for the customer segmentation.

In [ ]:
predictions_window_13 = pd.DataFrame(predictions_window_13, columns=[13],index=client_behaviour.index)
predictions_window_13.info()

In [ ]:
client_behaviour_extended = pd.merge(client_behaviour, predictions_window_13, left_index=True, right_index=True, how='outer')


In [ ]:
client_behaviour_extended

In [ ]:
client_dataset = pd.read_csv('/content/drive/MyDrive/IE/Capstone/client.csv',index_col=['client_id'])
client_dataset.head()

In [ ]:
new = client_dataset.merge(client_behaviour_extended[13], left_index=True, right_index=True)

In [ ]:
new.drop(columns ='Unnamed: 0',inplace=True)

In [ ]:
new = new.reset_index()
new.info()

In [ ]:
new.columns = ['client_id','first_purchase_date','last_purchase_date','last_invoice_date','recency','monetary','frequency', 'avg_days_between_purchases','most_common_order_channel','returns',
'product_id','unique_products',
'most_common_branch','total_quantity',
'purchase_interval_variance','lifespan',
'CLV','purchase_prediction']

In [ ]:
new['most_common_order_channel'].unique()

In [ ]:
new.to_csv('/content/drive/MyDrive/IE/Capstone/client_behaviour_churn.csv')

# Customer Segmentation

In this section, we merged the two datasets churn prediction and client information in order to create different clusters according to different purchase behaviour.

In [ ]:
#import client.csv from my drive
import pandas as pd
client = pd.read_csv('/content/drive/MyDrive/client_behaviour_churn.csv')
client.head()

In [ ]:
#Summary of the column CLV
client['CLV'].describe()

In [ ]:
#discretize the column CLV into 5, based on its quartiles
client['CLV'] = pd.qcut(client['CLV'], q=5, labels=['Very Low', 'Low', 'Medium','High','Very High'])
client['CLV'].value_counts()

In [ ]:
client['CLV'].value_counts()

In [ ]:
# prompt: Plot a HeatMap of the columns CLV and purchase_prediction
# Create a crosstab table of CLV and purchase_prediction
ct = pd.crosstab( client['purchase_prediction'],client['CLV'])

# Create a heatmap of the crosstab table
sns.heatmap(ct, annot=True, cmap="YlGnBu", fmt=".0f")
plt.ylabel("Purchase Prediction")
plt.xlabel("CLV")
plt.show()


In [ ]:
# prompt: Plot a heatmap with the columns CLV and most_common_order_channel
# Create a crosstab table of CLV and most_common_order_channel
ct = pd.crosstab( client['most_common_order_channel'],client['purchase_prediction'])

# Create a heatmap of the crosstab table
sns.heatmap(ct, annot=True, cmap="YlGnBu", fmt=".0f")
plt.ylabel("Most Common Order Channel")
plt.xlabel("Purchase Prediction")
plt.show()
